In [1]:
!pip install datasets tqdm pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.7 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00


In [1]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

/home/seang/Dev/Git/Amir/Grammar-Error-Correction/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!nvidia-smi

Tue Jul 25 16:20:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /home/seang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import random
import numpy as np
import torch
import datasets

In [4]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.cuda.set_device(0)

In [6]:
pd.set_option('display.max_colwidth', None)

In [7]:
!gdown "1aQSSoL7bCwBpCIWeiS22S97tsDmMajbB"

Downloading...
From (uriginal): https://drive.google.com/uc?id=1aQSSoL7bCwBpCIWeiS22S97tsDmMajbB
From (redirected): https://drive.google.com/uc?id=1aQSSoL7bCwBpCIWeiS22S97tsDmMajbB&confirm=t&uuid=af891b97-e439-49e0-983e-33428a4508d6
To: /home/seang/Dev/Git/Amir/Grammar-Error-Correction/C4_200M-part-0-of-9.csv
100%|██████████████████████████████████████| 5.19G/5.19G [01:07<00:00, 76.6MB/s]


In [7]:
df = pd.read_csv('./data/C4_200M-part-0-of-9.csv')
df.shape

(18386522, 2)

In [8]:
df.head()

,input,labels
0,"b'Bitcoin is for $7,094 this morning, which CoinDesk says.'","b'Bitcoin goes for $7,094 this morning, according to CoinDesk.'"
1,b'The effect of widespread dud targets two face up attack position monsters on the field.',"b'1. The effect of ""widespread dud"" targets two face up attack position monsters on the field.'"
2,"b'tax on sales of stores for non residents are set at 21% for 2014 and 20% in 2015 payable on sales tentatively earned from the difference of the property value some time of purchase (price differences according to working time) and theyear to which sale couples (sales costs), based on the approved annual on the base approved by law).'","b'Capital Gains tax on the sale of properties for non-residents is set at 21% for 2014 and 20% in 2015 payable on profits earned on the difference of the property value between the year of purchase (purchase price plus costs) and the year of sale (sales price minus costs), based on the approved annual percentage increase on the base value approved by law.'"
3,b'Much many brands and sellers still in the market.',b'Many brands and sellers still in the market.'
4,b'this is is the latest Maintenance release of Samba 3.6',b'This is is the latest maintenance release of Samba 3.6.'


In [9]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [10]:
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [11]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [16]:
train_df = df.sample(n=50000)
test_df = df.sample(n=5000)
train_df.shape, test_df.shape

((50000, 2), (5000, 2))

In [15]:
# from sklearn.model_selection import train_test_split
# train_df, test_df = train_test_split(df, test_size=0.05, shuffle=True)
# train_df.shape, test_df.shape

((17467195, 2), (919327, 2))

In [17]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

In [18]:
test_df.head()

,input,labels,input_token_len
1405291,b'How much time do your workers spend at lunch.',b'How much time do your workers spend at lunch?',15
7757367,b'Copyright \xc2\xa9 2019 Bullaburra Volunteer Bush Firebrigade All rights Reserved designed by JoomlArt.com. Joomla! it is Free Software was released under the GNU General Public License.',b'Copyright \xc2\xa9 2019 Bullaburra Volunteer Bush Fire Brigade. All Rights Reserved. Designed by JoomlArt.com. Joomla! is Free Software released under the GNU General Public License.',59
7035928,"b'Yes, the price is getting me reasonable.'","b'Yes, the price is getting reasonable.'",14
16725038,"b'The Royal Canadian Mint is full owner entity of the Canadian government and as a government entity, the RCM to guarantee the purity and every known authenticity of every gold coin that it produces.'","b'The Royal Canadian Mint is fully-owned entity of the Canadian Government and as a government entity, the RCM guarantees the purity and authenticity of every gold Maple Leaf coin that it produces.'",44
16864235,"b'Clint Gutherson (Sea Eagles), Valentine Holmes (Sharks), Jack Bird (Dragons), Euan Aitken (Dragons), Jake Mamo (Knight), Drewhisison (Dragons), Ryan Matterson (Eels), Matthew Lodge (Wests Tigers), Cameron McInnes (c) (Rabbitohs), Joe Ofahengaue (Broncos), Patric kMago (Raiders), Tevita Pangai (Raiders), Kelepi Tanginoa (Eels), Brett Greinke (Broncos), Willis Meehan (Roosters), Cameron Munster (Storm), Andy Saunders (Panthers), Jackson Hastings (Roosters).'","b'Clint Gutherson (Sea Eagles), Valentine Holmes (Sharks), Jack Bird (Dragons), Euan Aitken (Dragons), Jake Mamo (Knights), Drew Hutchison (Dragons), Ryan Matterson (Eels), Matthew Lodge (Wests Tigers), Cameron McInnes (c) (Rabbitohs), Joe Ofahengaue (Broncos), Patrick Mago (Raiders), Tevita Pangai (Raiders), Kelepi Tanginoa (Eels), Brett Greinke (Broncos), Willis Meehan (Roosters), Cameron Munster (Storm), Andy Saunders (Panthers), Jackson Hastings (Roosters).'",187


In [19]:
test_df["input_token_len"].describe()

count    5000.00000
mean       41.11460
std        30.21174
min         9.00000
25%        22.00000
50%        33.00000
75%        50.00000
max       546.00000
Name: input_token_len, dtype: float64

In [20]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [21]:
test_dataset

Dataset({
    features: ['input', 'labels', 'input_token_len', '__index_level_0__'],
    num_rows: 5000
})

In [23]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['labels']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [24]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 51
attention_mask 51
labels 46
{'input_ids': [3, 115, 31, 27776, 2, 31, 7, 1079, 4035, 9670, 1800, 8, 174, 3, 60, 7, 8399, 32, 17, 15, 16, 48, 625, 442, 13, 5199, 48, 1851, 16, 1163, 1220, 2145, 96, 155, 473, 114, 34, 2, 31, 7, 97, 12, 3, 60, 12484, 48, 1310, 541, 535, 31, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [3, 115, 31, 27776, 2, 31, 7, 1079, 26615, 1800, 8, 174, 12, 3, 60, 12484, 48, 625, 442, 45, 5199, 48, 1851, 30, 1163, 1220, 2145, 96, 196, 17, 4227, 114, 34, 2, 31, 7, 97, 12, 3, 60, 12484, 48, 541, 535, 31, 1]}


In [ ]:
!pip install rouge_score

In [25]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

/tmp/ipykernel_312720/2048908469.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


In [26]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [27]:
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="./data/c4_200m/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="./logs",
                        report_to="wandb")

In [28]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /home/seang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
# defining trainer using huggingface
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

In [ ]:
trainer.train()

/home/seang/Dev/Git/Amir/Grammar-Error-Correction/.venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: gec-nma-project (geccy). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


In [ ]:
trainer.train()

In [27]:
trainer.save_model('t5_gec_model')

In [ ]:
!zip -r 't5_gec_model.zip' 't5_gec_model'

In [ ]:
!rsync -rhPu --info=progress2 "/content/t5_gec_model.zip" "/content/drive/MyDrive/c4_200m"

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
model_name = 't5_gec_model'
torch_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
text = 'He are moving here.'
print(correct_grammar(text, num_return_sequences=2))